# Learning

In [9]:
from torchvision import datasets, models, transforms
import torch.optim as optim
import torch.nn as nn
from torchvision.transforms import *
from torch.utils.data import DataLoader
import torch
import numpy as np
from collections import defaultdict
import time
import copy

def train(dataloaders, model, criterion, optimizer, scheduler, device, num_epochs=20):
    def format_start_stop(start, stop):
        elapsed = stop - start
        return f'{elapsed//60:.0f}m {elapsed%50:.0f}s'
    
    best_model_weights = copy.deepcopy(model.state_dict())
    best_acc = -1.0
    
    loop_start = time.time()
    
    for epoch in range(num_epochs):
        train_loss, train_acc = 0.0, 0.0
        test_loss, test_acc = 0.0, 0.0
        train_start, test_start = 0.0, 0.0
        train_stop, test_stop = 0.0, 0.0
        
        for phase in ['train', 'test']:
            if phase == 'train':
                optimizer.step()
                scheduler.step()
                model.train()
                train_start = time.time()
            else:
                model.eval()
                test_start = time.time()

            running_loss = 0.0
            running_corrects = 0

            n = 0
            dataloader = dataloaders[phase]
            for inputs, labels in dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.mean() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                n += len(labels)

            epoch_loss = running_loss / float(n)
            epoch_acc = running_corrects.double() / float(n)
            
            if phase == 'train':
                train_stop = time.time()
                train_loss, train_acc = epoch_loss, epoch_acc
            else:
                test_stop = time.time()
                test_loss, test_acc = epoch_loss, epoch_acc
                
                if epoch_acc > best_acc:
                    best_model_weights = copy.deepcopy(model.state_dict())
                    best_acc = epoch_acc                
        
        train_time = format_start_stop(train_start, train_stop)
        test_time = format_start_stop(test_start, test_stop)
        
        train_metrics = f'TRAIN: {train_loss:.4f}, {train_acc:.4f}, {train_time}'
        test_metrics = f'TEST: {test_loss:.4f}, {test_acc:.4f}, {test_time}'
        print(f'epoch {str(epoch + 1).zfill(2)}/{str(num_epochs).zfill(2)} | {train_metrics} | {test_metrics}')
    
    loop_stop = time.time()
    loop_time = format_start_stop(loop_start, loop_stop)
    print(f'completed learning in {loop_time}, best accuracy {best_acc:.4f}')
    
    model.load_state_dict(best_model_weights)
    return model

def get_dataloaders():
    def clean_up(path):
        import os
        from shutil import rmtree
        
        ipynb_checkpoints = f'{path}/.ipynb_checkpoints'
        if os.path.exists(ipynb_checkpoints):
            rmtree(ipynb_checkpoints)
            
    def get_dataloader(phase):
        path = f'./shapes/{phase}'
        clean_up(path)
        
        transform = transforms.Compose([Resize(224), ToTensor()])
        image_folder = datasets.ImageFolder(path, transform=transform)
        # print(path, image_folder.classes)
        return DataLoader(image_folder, batch_size=4, shuffle=True, num_workers=4)
    
    return {phase: get_dataloader(phase) for phase in ['train', 'test', 'valid']}

np.random.seed(37)
torch.manual_seed(37)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
pretrained=True
num_classes = 3
num_epochs = 5

dataloaders = get_dataloaders()

model = models.resnet18(pretrained=pretrained)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Rprop(model.parameters(), lr=0.01)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1)

model = train(dataloaders, model, criterion, optimizer, scheduler, device, num_epochs=num_epochs)

epoch 01/05 | TRAIN: 1.1829, 0.6667, 0m 1s | TEST: 2.1690, 0.4000, 0m 0s
epoch 02/05 | TRAIN: 0.8402, 0.8000, 0m 1s | TEST: 0.0073, 1.0000, 0m 0s
epoch 03/05 | TRAIN: 0.7039, 0.8667, 0m 1s | TEST: 0.0217, 1.0000, 0m 0s
epoch 04/05 | TRAIN: 0.2520, 0.8667, 0m 1s | TEST: 0.0139, 1.0000, 0m 0s
epoch 05/05 | TRAIN: 0.4865, 0.8667, 0m 1s | TEST: 0.0039, 1.0000, 0m 0s
completed learning in 0m 5s, best accuracy 1.0000


In [11]:
def validate(model, dataloader):
    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            print(labels)
            print(preds)
            
validate(model, dataloaders['valid'])

tensor([2, 0, 1, 1], device='cuda:0')
tensor([2, 0, 1, 1], device='cuda:0')
tensor([0, 2, 1, 0], device='cuda:0')
tensor([0, 2, 1, 0], device='cuda:0')
tensor([2, 2, 2, 0], device='cuda:0')
tensor([2, 2, 2, 0], device='cuda:0')
tensor([1, 0, 1], device='cuda:0')
tensor([1, 0, 1], device='cuda:0')
